In [12]:
import findspark
findspark.init()


In [13]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("StudyApp").getOrCreate()

In [31]:
df=spark.read.csv("D:\employees.csv",header=True,inferSchema=True)
df.show()
df.count()

+---------+------+----------+-------------+------+------+----------------+--------------------+
|FirstName|Gender| StartDate|LastLoginTime|Salary|Bonus |SeniorManagement|                Team|
+---------+------+----------+-------------+------+------+----------------+--------------------+
|   Thomas|  Male| 3/31/1996|      6:53 AM| 61933|  4.17|            true|             Finance|
|    Maria|Female| 4/23/1993|     11:17 AM|  5689|11.858|           false|             Finance|
|    Larry|  Male| 1/24/1998|      4:47 PM|  9894| 1.389|            true|               Legal|
|   Dennis|  Male| 4/18/1987|      1:35 AM|115163|10.125|           false|               Legal|
|     Ruby|Female| 4/18/1988|      2:35 AM| 65476|10.012|            true|             Product|
|     Ruby|Female| 7/20/2015|     10:43 AM| 65476|10.012|            true|             Finance|
|   Angela|Female| 7/20/2016|      6:29 AM| 95570|10.012|            true|         Engineering|
|  Frances|Female| 7/20/2017|      6:51 

998

In [32]:
df.printSchema()
df.select('Team').distinct().show()
#df.columns[:4]

root
 |-- FirstName: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- StartDate: string (nullable = true)
 |-- LastLoginTime: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Bonus : double (nullable = true)
 |-- SeniorManagement: boolean (nullable = true)
 |-- Team: string (nullable = true)

+--------------------+
|                Team|
+--------------------+
|               Sales|
|         Engineering|
|Business Development|
|             Finance|
|     Client Services|
|        Distribution|
|               Legal|
|           Marketing|
|             Product|
|     Human Resources|
+--------------------+



In [33]:
#formating
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.feature import StringIndexer


In [63]:
va=VectorAssembler(inputCols=['Salary'],outputCol='InputFeature')
indexer=StringIndexer(inputCol='Team',outputCol='Team_num')#conver into integer
df1=indexer.fit(df).transform(df)
df2=va.transform(df1)
df2.show(5)

+---------+------+---------+-------------+------+------+----------------+-------+--------+------------+
|FirstName|Gender|StartDate|LastLoginTime|Salary|Bonus |SeniorManagement|   Team|Team_num|InputFeature|
+---------+------+---------+-------------+------+------+----------------+-------+--------+------------+
|   Thomas|  Male|3/31/1996|      6:53 AM| 61933|  4.17|            true|Finance|     2.0|   [61933.0]|
|    Maria|Female|4/23/1993|     11:17 AM|  5689|11.858|           false|Finance|     2.0|    [5689.0]|
|    Larry|  Male|1/24/1998|      4:47 PM|  9894| 1.389|            true|  Legal|     5.0|    [9894.0]|
|   Dennis|  Male|4/18/1987|      1:35 AM|115163|10.125|           false|  Legal|     5.0|  [115163.0]|
|     Ruby|Female|4/18/1988|      2:35 AM| 65476|10.012|            true|Product|     3.0|   [65476.0]|
+---------+------+---------+-------------+------+------+----------------+-------+--------+------------+
only showing top 5 rows



In [46]:
finaldata=df2.select('InputFeature',"Team_num")
finaldata.show(5)

+------------+--------+
|InputFeature|Team_num|
+------------+--------+
|   [61933.0]|     2.0|
|    [5689.0]|     2.0|
|    [9894.0]|     5.0|
|  [115163.0]|     5.0|
|   [65476.0]|     3.0|
+------------+--------+
only showing top 5 rows



In [47]:
train,test=finaldata.randomSplit([0.70,0.30])

In [48]:
from pyspark.ml.classification import DecisionTreeClassifier
dtcmodel=DecisionTreeClassifier(labelCol="Team_num",featuresCol="InputFeature")
model=dtcmodel.fit(train)

In [49]:
model

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_6e7af4734286, depth=5, numNodes=29, numClasses=10, numFeatures=1

In [50]:
prediction_result=model.transform(test)

In [51]:
prediction_result.show()

+------------+--------+--------------------+--------------------+----------+
|InputFeature|Team_num|       rawPrediction|         probability|prediction|
+------------+--------+--------------------+--------------------+----------+
|   [35013.0]|     3.0|[18.0,16.0,16.0,1...|[0.13533834586466...|       0.0|
|   [35477.0]|     8.0|[18.0,16.0,16.0,1...|[0.13533834586466...|       0.0|
|   [35575.0]|     9.0|[18.0,16.0,16.0,1...|[0.13533834586466...|       0.0|
|   [35633.0]|     9.0|[18.0,16.0,16.0,1...|[0.13533834586466...|       0.0|
|   [35884.0]|     6.0|[18.0,16.0,16.0,1...|[0.13533834586466...|       0.0|
|   [36643.0]|     4.0|[18.0,16.0,16.0,1...|[0.13533834586466...|       0.0|
|   [36697.0]|     6.0|[18.0,16.0,16.0,1...|[0.13533834586466...|       0.0|
|   [36837.0]|     4.0|[18.0,16.0,16.0,1...|[0.13533834586466...|       0.0|
|   [36946.0]|     7.0|[18.0,16.0,16.0,1...|[0.13533834586466...|       0.0|
|   [37076.0]|     1.0|[18.0,16.0,16.0,1...|[0.13533834586466...|       0.0|

In [53]:
prediction_result.select('InputFeature','Team_num','Prediction').show()

+------------+--------+----------+
|InputFeature|Team_num|Prediction|
+------------+--------+----------+
|   [35013.0]|     3.0|       0.0|
|   [35477.0]|     8.0|       0.0|
|   [35575.0]|     9.0|       0.0|
|   [35633.0]|     9.0|       0.0|
|   [35884.0]|     6.0|       0.0|
|   [36643.0]|     4.0|       0.0|
|   [36697.0]|     6.0|       0.0|
|   [36837.0]|     4.0|       0.0|
|   [36946.0]|     7.0|       0.0|
|   [37076.0]|     1.0|       0.0|
|   [37919.0]|     5.0|       0.0|
|   [38078.0]|     3.0|       0.0|
|   [38275.0]|     9.0|       0.0|
|   [38375.0]|     5.0|       0.0|
|   [38712.0]|     1.0|       0.0|
|   [39833.0]|     7.0|       0.0|
|   [40119.0]|     8.0|       0.0|
|   [40341.0]|     2.0|       0.0|
|   [40837.0]|     1.0|       0.0|
|   [41126.0]|     6.0|       0.0|
+------------+--------+----------+
only showing top 20 rows



In [56]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator=MulticlassClassificationEvaluator(labelCol="Team_num",predictionCol='prediction')
accuracy=evaluator.evaluate(prediction_result)
print("Accurancy",accuracy)
print("Test error of model",(1-accuracy))

Accurancy 0.08894789246306493
Test error of model 0.9110521075369351


In [59]:
from pyspark.ml.classification import RandomForestClassifier
dtcmodel=RandomForestClassifier(labelCol="Team_num",featuresCol="InputFeature")
model=dtcmodel.fit(train)


In [60]:
prediction_result=model.transform(test)
prediction_result.show()

+------------+--------+--------------------+--------------------+----------+
|InputFeature|Team_num|       rawPrediction|         probability|prediction|
+------------+--------+--------------------+--------------------+----------+
|   [35013.0]|     3.0|[1.79152548885956...|[0.08957627444297...|       8.0|
|   [35477.0]|     8.0|[1.79152548885956...|[0.08957627444297...|       8.0|
|   [35575.0]|     9.0|[1.79152548885956...|[0.08957627444297...|       8.0|
|   [35633.0]|     9.0|[1.79152548885956...|[0.08957627444297...|       8.0|
|   [35884.0]|     6.0|[1.79152548885956...|[0.08957627444297...|       8.0|
|   [36643.0]|     4.0|[1.79152548885956...|[0.08957627444297...|       8.0|
|   [36697.0]|     6.0|[1.79152548885956...|[0.08957627444297...|       8.0|
|   [36837.0]|     4.0|[1.79152548885956...|[0.08957627444297...|       8.0|
|   [36946.0]|     7.0|[1.79152548885956...|[0.08957627444297...|       8.0|
|   [37076.0]|     1.0|[1.79152548885956...|[0.08957627444297...|       8.0|

In [61]:
evaluator=MulticlassClassificationEvaluator(labelCol="Team_num",predictionCol='prediction')
accuracy=evaluator.evaluate(prediction_result)
print("Accurancy",accuracy)
print("Test error model",(1-accuracy))

Accurancy 0.08682118197787979
Test error model 0.9131788180221202
